# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs
from bokeh.io import push_notebook,show,output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()
import requests

# Import API key
from api_keys import geoapify_key

Loading BokehJS ...

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,50.81,59,10,13.20,NZ,1670296509
1,1,aktash,50.3000,87.7333,-11.09,87,9,4.05,RU,1670296510
2,2,mar del plata,-38.0023,-57.5575,69.82,73,0,10.36,AR,1670296511
3,3,muli,2.9167,73.5667,80.51,75,99,8.25,MV,1670296512
4,4,rawson,-43.3002,-65.1023,71.17,35,20,18.63,AR,1670296513


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", size= "Humidity", legend= 'right', c= "City"
)

# Display the map plot_1
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
narrowed_city_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 10) & (city_data_df["Cloudiness"] == 0) & \
                                   (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80)].dropna()

narrowed_city_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,tura,25.5198,90.2201,70.09,63,0,1.34,IN,1670296305
74,74,busselton,-33.6500,115.3333,72.14,38,0,9.60,AU,1670296560
150,150,presidencia roque saenz pena,-26.7852,-60.4388,76.39,43,0,6.53,AR,1670296620
156,156,nacogdoches,31.6002,-94.6335,70.05,98,0,8.05,US,1670296603
275,275,la orilla,17.9833,-102.2333,76.23,86,0,1.95,MX,1670296721
440,440,cartagena,10.3997,-75.5144,78.42,83,0,4.61,CO,1670296842
498,498,alice,27.7523,-98.0697,71.33,95,0,9.22,US,1670296869
517,517,tecoanapa,16.5167,-98.7500,77.52,77,0,4.47,MX,1670296878


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng", "Humidity"]]


hotel_df["Hotel Name"] = ""

# Display the sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
54,tura,IN,25.5198,90.2201,63,
74,busselton,AU,-33.6500,115.3333,38,
150,presidencia roque saenz pena,AR,-26.7852,-60.4388,43,
156,nacogdoches,US,31.6002,-94.6335,98,
275,la orilla,MX,17.9833,-102.2333,86,
440,cartagena,CO,10.3997,-75.5144,83,
498,alice,US,27.7523,-98.0697,95,
517,tecoanapa,MX,16.5167,-98.7500,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
categories= "accommodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    long = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
tura - nearest hotel: No hotel found
busselton - nearest hotel: Broadwater Beach Resort
presidencia roque saenz pena - nearest hotel: Hotel Presidente
nacogdoches - nearest hotel: No hotel found
la orilla - nearest hotel: Victoria
cartagena - nearest hotel: Hotel Bellamar
alice - nearest hotel: No hotel found
tecoanapa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
54,tura,IN,25.5198,90.2201,63,No hotel found
74,busselton,AU,-33.6500,115.3333,38,Broadwater Beach Resort
150,presidencia roque saenz pena,AR,-26.7852,-60.4388,43,Hotel Presidente
156,nacogdoches,US,31.6002,-94.6335,98,No hotel found
275,la orilla,MX,17.9833,-102.2333,86,Victoria
440,cartagena,CO,10.3997,-75.5144,83,Hotel Bellamar
498,alice,US,27.7523,-98.0697,95,No hotel found
517,tecoanapa,MX,16.5167,-98.7500,77,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [50]:
%%capture --no-display


# Configure the map plot
map_plot_2 =hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    hover_cols= ["Country","Hotel Name"],
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)